# Design of `qsirecon`

This document describes how data is read by and processed by `qsirecon`, which takes the output from `qsiprep` and performs reconstruction/tracking/etc on these data. 

## How does pybids deal with derivatives?

The output from `qsiprep` goes into a directory that mirrors the input data. We need to get the masks, bvals, bvecs dwi data, anatomical data, transforms and tissue probability maps from the `qsiprep` output. Here is a typical output from a `qsiprep` run (omitting figures):

```
qsiprep/sub-abcd.html
qsiprep/sub-abcd
qsiprep/sub-abcd/dwi
qsiprep/sub-abcd/dwi/sub-abcd_space-T1w_desc-brain_mask.nii.gz
qsiprep/sub-abcd/dwi/sub-abcd_space-T1w_desc-preproc.bvec
qsiprep/sub-abcd/dwi/sub-abcd_space-MNI152NLin2009cAsym_desc-preproc_dwi.nii.gz
qsiprep/sub-abcd/dwi/sub-abcd_space-T1w_bvec.nii.gz
qsiprep/sub-abcd/dwi/sub-abcd_space-MNI152NLin2009cAsym_dwiref.nii.gz
qsiprep/sub-abcd/dwi/sub-abcd_confounds.tsv
qsiprep/sub-abcd/dwi/sub-abcd_space-MNI152NLin2009cAsym_b0series.nii.gz
qsiprep/sub-abcd/dwi/sub-abcd_space-T1w_desc-preproc.bval
qsiprep/sub-abcd/dwi/sub-abcd_space-T1w_b0series.nii.gz
qsiprep/sub-abcd/dwi/sub-abcd_space-MNI152NLin2009cAsym_desc-preproc.bvec
qsiprep/sub-abcd/dwi/sub-abcd_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz
qsiprep/sub-abcd/dwi/sub-abcd_space-MNI152NLin2009cAsym_bvec.nii.gz
qsiprep/sub-abcd/dwi/sub-abcd_space-T1w_dwiref.nii.gz
qsiprep/sub-abcd/dwi/sub-abcd_space-T1w_desc-preproc.nii.gz
qsiprep/sub-abcd/dwi/sub-abcd_space-MNI152NLin2009cAsym_desc-preproc.bval
qsiprep/sub-abcd/anat
qsiprep/sub-abcd/anat/sub-abcd_dseg.nii.gz
qsiprep/sub-abcd/anat/sub-abcd_space-MNI152NLin2009cAsym_dseg.nii.gz
qsiprep/sub-abcd/anat/sub-abcd_space-MNI152NLin2009cAsym_label-CSF_probseg.nii.gz
qsiprep/sub-abcd/anat/sub-abcd_label-WM_probseg.nii.gz
qsiprep/sub-abcd/anat/sub-abcd_space-MNI152NLin2009cAsym_label-WM_probseg.nii.gz
qsiprep/sub-abcd/anat/sub-abcd_from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5
qsiprep/sub-abcd/anat/sub-abcd_space-MNI152NLin2009cAsym_label-GM_probseg.nii.gz
qsiprep/sub-abcd/anat/sub-abcd_label-GM_probseg.nii.gz
qsiprep/sub-abcd/anat/sub-abcd_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz
qsiprep/sub-abcd/anat/sub-abcd_desc-brain_mask.nii.gz
qsiprep/sub-abcd/anat/sub-abcd_from-MNI152NLin2009cAsym_to-T1w_mode-image_xfm.h5
qsiprep/sub-abcd/anat/sub-abcd_label-CSF_probseg.nii.gz
qsiprep/sub-abcd/anat/sub-abcd_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz
qsiprep/sub-abcd/anat/sub-abcd_from-orig_to-T1w_mode-image_xfm.txt
qsiprep/sub-abcd/anat/sub-abcd_desc-preproc_T1w.nii.gz
qsiprep/dataset_description.json
```

In [1]:
from bids.layout import BIDSLayout

qsiprep_output = "/home/matt/projects/qsiprep/scratch/ef_test/qsiprep"
orig_bids = "/home/matt/projects/test_bids_data/ef_test"

qp_layout = BIDSLayout(qsiprep_output)

# Get the preprocessed dwi niftis
dwi_files = [f.filename for f in qp_layout.get(type='dwi', extensions=['nii', 'nii.gz'])]

print(dwi_files)

ModuleNotFoundError: No module named 'bids'

We see there is one output in MNI space and another in T1w space. We only want to work on the one in T1w for this pipeline

In [2]:
%%writefile pipeline.json
{
  "name": "dsistudio_pipeline",
  "space": "T1w",
  "atlases": ["schaefer100", "schaefer200"],
  "nodes": [
    {
      "name": "dsistudio_gqi",
      "software": "DSI Studio",
      "action": "reconstruction",
      "input": "qsiprep",
      "output": ["fibgz"],
      "parameters": {"method": "gqi"}
    },
    {
      "name": "scalar_export",
      "software": "DSI Studio",
      "action": "export",
      "input": "dsistudio_gqi",
      "output": ["gfa"],
      "parameters": {}
    },
    {
      "name": "streamline_connectivity",
      "software": "DSI Studio",
      "action": "connectivity",
      "input": "dsistudio_gqi",
      "output": [
        "connectivity"
      ],
      "parameters": {
        "turning_angle": 35,
        "method": 0,
        "smoothing": 0.0,
        "step_size": 1.0,
        "min_length": 10,
        "max_length": 250,
        "seed_plan": 0,
        "interpolation": 0,
        "initial_dir": 2, 
        "fiber_count": 5000000,
        "connectivity_value": "count,ncount,mean_length,gfa",
        "connectivity_type": "pass,end",
        "output_trk": "no_trk"
      }
    },
    {
      "name": "controlability",
      "input": "streamline_connectivity",
      "action": "controllability",
      "output": ["summary"]
    }
  ]
}

Overwriting pipeline.json


In [3]:
import json
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as niu
import logging
import os
from qsiprep.interfaces.dsi_studio import (DSIStudioCreateSrc, DSIStudioGQIReconstruction,
                                           DSIStudioAtlasGraph, DSIStudioExport)
from qsiprep.interfaces.bids import QsiprepOutput, DerivativesDataSink
from qsiprep.interfaces.utils import GetConnectivityAtlases
from qsiprep.interfaces.connectivity import Controllability

LOGGER = logging.getLogger('nipype.interface')
json_file = "pipeline.json"
with open(json_file, "r") as f:
    spec = json.load(f)

work_space = spec["space"]
dwi_files = [f for f in dwi_files if 'space-' + work_space in f]
print(dwi_files)

/Users/mcieslak/miniconda3/envs/qsiprep/lib/python3.6/site-packages/skimage/__init__.py:80: ResourceWarning: unclosed file <_io.TextIOWrapper name='/Users/mcieslak/miniconda3/envs/qsiprep/lib/python3.6/site-packages/pytest.py' mode='r' encoding='utf-8'>
  imp.find_module('pytest')
/Users/mcieslak/miniconda3/envs/qsiprep/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/mcieslak/miniconda3/envs/qsiprep/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/mcieslak/miniconda3/envs/qsiprep/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


['/Users/mcieslak/projects/qsiprep/scratch/abcd_test/test_output/qsiprep/sub-abcd/dwi/sub-abcd_space-T1w_desc-preproc_dwi.nii.gz']


In [4]:
inputnode = pe.Node(niu.IdentityInterface(fields=['dwi_file']),
                    name="inputnode")
inputnode.iterables = [('dwi_file', dwi_files)]

# Get bvals, bvecs
grab_schemes = pe.Node(QsiprepOutput(), name="grab_schemes")
# Prepare atlases for analysis
atlas_names = spec['atlases']
get_atlases = pe.Node(GetConnectivityAtlases(atlas_names=atlas_names), name="get_atlases")

Here are some functions that create the remaining nodes and connect them to the scheme grabber node.

In [5]:
qsiprep_outputs = QsiprepOutput().output_spec.class_editable_traits()

def init_dsi_studio_recon_wf(space, method="gqi", name="dsi_studio_recon"):
    inputnode = pe.Node(
        niu.IdentityInterface(
            fields=qsiprep_outputs),
        name="inputnode")
    outputnode = pe.Node(
        niu.IdentityInterface(
            fields=['fibgz']),
        name="outputnode")
    workflow = pe.Workflow(name=name)
    create_src = pe.Node(DSIStudioCreateSrc(), name="create_src")
    gqi_recon = pe.Node(DSIStudioGQIReconstruction(), name="gqi_recon")
    
    # Save the output in the outputs directory
    ds_gqi_fibgz = pe.Node(DerivativesDataSink(
                                space=space,
                                extension='.fib.gz',
                                compress=True),
                            name='ds_gqi_fibgz',
                            run_without_submitting=True)
    workflow.connect([
        (inputnode, create_src, [('dwi_file', 'input_nifti_file'),
                                 ('bval_file', 'input_bvals_file'),
                                 ('bvec_file', 'input_bvecs_file')]),
        (create_src, gqi_recon, [('output_src', 'input_src_file')]),
        (gqi_recon, outputnode, [('output_fib', 'fibgz')]),
        (inputnode, ds_gqi_fibgz, [('dwi_file', 'source_file')]),
        (gqi_recon, ds_gqi_fibgz, [('output_fib', 'in_file')])
    ])
    return workflow

def init_dsi_studio_connectivity_workflow(name="dsi_studio_connectivity", n_procs=1,
                                          **kwargs):
    inputnode = pe.Node(
    niu.IdentityInterface(
            fields=qsiprep_outputs + ['fibgz', 'atlas_configs']),
        name="inputnode")
    outputnode = pe.Node(niu.IdentityInterface(fields=['matfile']),
                         name="outputnode")
    workflow = pe.Workflow(name=name)
    calc_connectivity = pe.Node(DSIStudioAtlasGraph(n_procs=n_procs, **kwargs), 
                                name='calc_connectivity')
    workflow.connect([
        (inputnode, calc_connectivity, [('atlas_configs', 'atlas_configs'),
                                        ('fibgz', 'input_fib')]),
        (calc_connectivity, outputnode, [('connectivity_matfile', 'matfile')])
    ])
    return workflow


def init_dsi_studio_export_workflow(space, name="dsi_studio_export"):
    inputnode = pe.Node(
    niu.IdentityInterface(
            fields=qsiprep_outputs + ['fibgz']),
        name="inputnode")
    outputnode = pe.Node(
        niu.IdentityInterface(fields=['gfa', 'fa0', 'fa1', 'fa2', 'fa3', 'iso']),
        name="outputnode")
    workflow = pe.Workflow(name=name)
    export = pe.Node(DSIStudioExport(to_export="gfa,fa0,fa1,fa2,fa3,iso"), name='export')
    
    workflow.connect([
        (inputnode, export, [('fibgz', 'input_file')]),
        (export, outputnode, [('gfa_file', 'gfa'), ('fa0_file', 'fa0'), ('fa1_file', 'fa1'),
                              ('fa2_file', 'fa2'), ('fa3_file', 'fa3'), ('iso_file', 'iso')])
    ])
    
    return workflow


def init_controllability_workflow(name="controllability"):
    inputnode = pe.Node(niu.IdentityInterface(fields=qsiprep_outputs + ['matfile']),
                        name="inputnode")
    outputnode = pe.Node(
        niu.IdentityInterface(fields=['matfile']),
        name="outputnode")
    
    calc_control = pe.Node(Controllability(), name='calc_control')
    workflow = pe.Workflow(name=name)
    workflow.connect([
        (inputnode, calc_control, [('matfile', 'matfile')]),
        (calc_control, outputnode, [('controllability', 'matfile')])
    ])
    return workflow

def workflow_from_spec(node_spec, space):
    software = node_spec.get("software", "qsiprep")
    if software == "DSI Studio":
        if node_spec["action"] == "reconstruction":
            return init_dsi_studio_recon_wf(name=node_spec["name"],space=space,
                                            **node_spec["parameters"])
        if node_spec["action"] == "export":
            return init_dsi_studio_export_workflow(name=node_spec["name"], space=space)
        
        if node_spec["action"] == "connectivity":
            return init_dsi_studio_connectivity_workflow(name=node_spec["name"], n_procs=1,
                                          **node_spec["parameters"])
    else:
        if node_spec['action'] == "controllability":
            return init_controllability_workflow(name=node_spec["name"])

        
# Create a workflow
workflow_name = spec['name']
workflow = pe.Workflow(name=workflow_name)
workflow.base_dir = os.getcwd()
workflow.connect([(inputnode, grab_schemes, [('dwi_file', 'in_file')]),
                  (inputnode, get_atlases, [('dwi_file', 'reference_image')])
                 ])

space = spec['space']
if space == "T1w":
    workflow.connect([
        (grab_schemes, get_atlases, [('t1_2_mni_reverse_transform', 'forward_transform')])
    ])

# First pass: add the workflows
for node_spec in spec["nodes"]:
    workflow.add_nodes([workflow_from_spec(node_spec, space=space)])

Now we need to connect the inputs and outputs from each of the nodes

In [6]:
default_connections = [(trait, trait) for trait in qsiprep_outputs]
default_input_set = set(qsiprep_outputs)

def get_connections(src, dest):
    src_outputs = set(src.outputs.get().keys())
    dest_inputs = set(dest.inputs.get().keys())
    overlap = src_outputs.intersection(dest_inputs) - default_input_set
    return [(trait, trait) for trait in overlap]


for node_spec in spec['nodes']:
    
    # get the nipype node object
    node_name = node_spec['name'] + ".inputnode"
    node = workflow.get_node(node_name)
    
    # directly connect all the qsiprep outputs to every node
    workflow.connect([(grab_schemes, node, default_connections)])

    # connect the outputs from the upstream node to this node
    if not node_spec['input'] == 'qsiprep':
        upstream_node_name = node_spec['input'] + '.outputnode'
        upstream_node = workflow.get_node(upstream_node_name)
        # Connect outputs from the inputnode to spec of this node
        connections = get_connections(upstream_node, node)
        print("connecting",(upstream_node, node, connections))
        workflow.connect([(upstream_node, node, connections)])

    # If it's a connectivity calculation, send it the atlas configs
    if node_spec['action'] == 'connectivity':
        workflow.connect([(get_atlases, node, 
                           [('atlas_configs', 'atlas_configs')])])
    
workflow.config['execution']['stop_on_first_crash'] = 'true'
workflow.config['execution']['remove_unnecessary_outputs'] = 'false'
workflow.run()

connecting (dsistudio_gqi.outputnode, scalar_export.inputnode, [('fibgz', 'fibgz')])
connecting (dsistudio_gqi.outputnode, streamline_connectivity.inputnode, [('fibgz', 'fibgz')])
connecting (streamline_connectivity.outputnode, controlability.inputnode, [('matfile', 'matfile')])
190106-22:07:59,844 nipype.workflow INFO:
	 Workflow dsistudio_pipeline settings: ['check', 'execution', 'logging', 'monitoring']
190106-22:07:59,862 nipype.workflow INFO:
	 Running serially.
190106-22:07:59,863 nipype.workflow INFO:
	 [Node] Setting-up "dsistudio_pipeline.grab_schemes" in "/Users/mcieslak/projects/qsiprep/notebooks/dsistudio_pipeline/_dwi_file_..Users..mcieslak..projects..qsiprep..scratch..abcd_test..test_output..qsiprep..sub-abcd..dwi..sub-abcd_space-T1w_desc-preproc_dwi.nii.gz/grab_schemes".
190106-22:07:59,866 nipype.workflow INFO:
	 [Node] Outdated cache found for "dsistudio_pipeline.grab_schemes".
190106-22:07:59,881 nipype.workflow INFO:
	 [Node] Running "grab_schemes" ("qsiprep.interfac

190106-22:08:10,324 nipype.workflow INFO:
	 [Node] Finished "dsistudio_pipeline.dsistudio_gqi.gqi_recon".
190106-22:08:10,325 nipype.workflow INFO:
	 [Node] Setting-up "dsistudio_pipeline.scalar_export.export" in "/Users/mcieslak/projects/qsiprep/notebooks/dsistudio_pipeline/scalar_export/_dwi_file_..Users..mcieslak..projects..qsiprep..scratch..abcd_test..test_output..qsiprep..sub-abcd..dwi..sub-abcd_space-T1w_desc-preproc_dwi.nii.gz/export".
190106-22:08:10,328 nipype.workflow INFO:
	 [Node] Outdated cache found for "dsistudio_pipeline.scalar_export.export".
190106-22:08:10,337 nipype.workflow INFO:
	 [Node] Running "export" ("qsiprep.interfaces.dsi_studio.DSIStudioExport"), a CommandLine Interface with command:
dsi_studio --action=exp --source=/Users/mcieslak/projects/qsiprep/notebooks/dsistudio_pipeline/scalar_export/_dwi_file_..Users..mcieslak..projects..qsiprep..scratch..abcd_test..test_output..qsiprep..sub-abcd..dwi..sub-abcd_space-T1w_desc-preproc_dwi.nii.gz/export/sub-abcd_spac

190106-22:11:15,71 nipype.workflow INFO:
	 [Node] Finished "dsistudio_pipeline.streamline_connectivity.calc_connectivity".
190106-22:11:15,73 nipype.workflow INFO:
	 [Node] Setting-up "dsistudio_pipeline.controlability.calc_control" in "/Users/mcieslak/projects/qsiprep/notebooks/dsistudio_pipeline/controlability/_dwi_file_..Users..mcieslak..projects..qsiprep..scratch..abcd_test..test_output..qsiprep..sub-abcd..dwi..sub-abcd_space-T1w_desc-preproc_dwi.nii.gz/calc_control".
190106-22:11:15,77 nipype.workflow INFO:
	 [Node] Outdated cache found for "dsistudio_pipeline.controlability.calc_control".
190106-22:11:15,83 nipype.workflow INFO:
	 [Node] Running "calc_control" ("qsiprep.interfaces.connectivity.Controllability")


/Users/mcieslak/miniconda3/envs/qsiprep/lib/python3.6/site-packages/scipy/io/matlab/miobase.py:414: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  arr[empties] = ' '


190106-22:11:15,996 nipype.interface INFO:
	 writing /Users/mcieslak/projects/qsiprep/notebooks/dsistudio_pipeline/controlability/_dwi_file_..Users..mcieslak..projects..qsiprep..scratch..abcd_test..test_output..qsiprep..sub-abcd..dwi..sub-abcd_space-T1w_desc-preproc_dwi.nii.gz/calc_control/combined_connectivity_controllability.mat
190106-22:11:16,9 nipype.workflow INFO:
	 [Node] Finished "dsistudio_pipeline.controlability.calc_control".
